In [21]:
#Importação das bibliotecas necessárias

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [22]:
# 1. Gerar dados simulados (ou substitua por pd.read_csv('arquivo.csv'))
dados = pd.read_csv('store.csv')
print("Dados carregados com sucesso!")
print(dados.head())

print("\n📋 Colunas do dataset:")
print(dados.columns)


Dados carregados com sucesso!
   Store StoreType Assortment  CompetitionDistance  CompetitionOpenSinceMonth  \
0      1         c          a               1270.0                        9.0   
1      2         a          a                570.0                       11.0   
2      3         a          a              14130.0                       12.0   
3      4         c          c                620.0                        9.0   
4      5         a          a              29910.0                        4.0   

   CompetitionOpenSinceYear  Promo2  Promo2SinceWeek  Promo2SinceYear  \
0                    2008.0       0              NaN              NaN   
1                    2007.0       1             13.0           2010.0   
2                    2006.0       1             14.0           2011.0   
3                    2009.0       0              NaN              NaN   
4                    2015.0       0              NaN              NaN   

     PromoInterval  
0              NaN  
1 

In [23]:
#Engenharia de atributos
# Como o arquivo store.csv não tem vendas, vamos criar um valor fictício com base em outras colunas
if 'Promo' in dados.columns:
    dados['Sales'] = (dados['Promo'].fillna(0) * 3000) + np.random.randint(5000, 20000, size=len(dados))
else:
    dados['Sales'] = np.random.randint(5000, 20000, size=len(dados))

In [27]:
#Garantir que todas as colunas sejam numéricas
for c in dados.columns:
    dados[c] = pd.to_numeric(dados[c], errors='coerce').fillna(0)



In [28]:
#Dividir em treino e teste
X = dados.drop('Sales', axis=1)
y = dados['Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
# Modelos simples
modelos = {
    "Regressão Linear": LinearRegression(),
    "Árvore de Decisão": DecisionTreeRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42, n_estimators=100, verbosity=0)
}


In [32]:
resultados = {}
for nome, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    # Remove valores NaN e garante mesmo tamanho
    y_pred = np.nan_to_num(y_pred)
    y_test_limpo = np.nan_to_num(y_test[:len(y_pred)])

    rmse = np.sqrt(mean_squared_error(y_test_limpo, y_pred))
    mae = mean_absolute_error(y_test_limpo, y_pred)
    r2 = r2_score(y_test_limpo, y_pred)

    resultados[nome] = {'RMSE': rmse, 'MAE': mae, 'R²': r2}

In [35]:
#Mostrar resultados
metricas = pd.DataFrame(resultados).T
print("Desempenho dos modelos:")
print(metricas)

Desempenho dos modelos:
                          RMSE          MAE        R²
Regressão Linear   4223.699227  3634.031429 -0.005953
Árvore de Decisão  6388.390156  5096.869955 -1.301307
XGBoost            5162.783552  4238.097168 -0.503002


In [36]:
#Melhor modelo
melhor = metricas['RMSE'].idxmin()
print(f"Melhor modelo: {melhor}")

Melhor modelo: Regressão Linear


In [39]:
#Importância das variáveis (para árvore ou XGBoost)
modelo_final = modelos[melhor]
if hasattr(modelo_final, 'feature_importances_'):
    importancias = pd.Series(modelo_final.feature_importances_, index=X.columns).sort_values(ascending=False)
    print("Importância das variáveis:")
    print(importancias)
else:
    print(f"O modelo escolhido {melhor} não calcula importância de variáveis.")


O modelo escolhido Regressão Linear não calcula importância de variáveis.
